# Imports

In [16]:
import os
import glob
import polars as pl

import datetime
# from datetime import date, time, timedelta, datetime
from datetime import time

import re
import pickle
import json

import pandas_market_calendars as mcal

import importlib
import config
importlib.reload(config)

# import time
from time import sleep

from IPython.display import display, Markdown

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJSTickFormatter, DatetimeTickFormatter, DatetimeTicker, HoverTool,CustomJSHover, LinearAxis, Range1d
from bokeh.io import output_notebook
from bokeh.io import output_file, save, show, reset_output

from theta_data_v_3_client import client

from theta_data_v_3_client.api.stock.stock_snapshot_trade import sync as stock_snapshot_trade
from theta_data_v_3_client.api.option.option_list_contracts import sync as option_list_contracts
 

In [2]:
SYMBOL = "AAPL"
START_DATE = datetime.date(2025, 8, 20)
END_DATE = datetime.date(2025, 8, 21)
# TEST_INTERVAL = StockHistoryOhlcInterval.VALUE_13
# TEST_FORMAT_EOD = StockHistoryEodFormat.NDJSON
# TEST_FORMAT_OHLC = StockHistoryOhlcFormat.NDJSON
START_TIME = datetime.time(9, 30)
END_TIME = datetime.time(16, 0)
TEST_TIME="09:30:01.000"
EXPIRATION_DATE = datetime.date(2025, 8, 22)
INDEX_SYMBOL="SPX"

In [3]:
# Example: create an API client instance
base_url = "http://localhost:25504/v3"  # Change to your API's base URL

client1 = client.Client(base_url=base_url)
 
# Example: perform a simple API call (replace with an actual endpoint/method)
try:
    # http://localhost:25503/v3/stock/snapshot/trade?symbol=AAPL
    response = stock_snapshot_trade(client=client1, symbol="AAPL")
    print(f"Client initialized successfully. {response}")
except Exception as e:
    print(f"Error initializing or calling client: {e}")

Client initialized successfully. timestamp,symbol,sequence,size,condition,price
2025-08-27T06:52:01.36,"AAPL",189384,10,1,229.20



In [4]:
base_url_1 = "http://localhost:25503/v3"  # Change to your API's base URL

client2 = client.Client(base_url=base_url_1)
 
# Example: perform a simple API call (replace with an actual endpoint/method)
try:
    # http://localhost:25504/v3/option/list/contracts?symbol=AAPL
    response = option_list_contracts(client=client2, request_type="trade", symbol="AAPL", date=START_DATE)
    print(f"Client initialized successfully. {response}")
except Exception as e:
    print(f"Error initializing or calling client: {e}")

Client initialized successfully. symbol,expiration,strike,right
"AAPL","2025-08-29",215.000,"PUT"
"AAPL","2025-08-29",215.000,"CALL"
"AAPL","2025-09-12",215.000,"PUT"
"AAPL","2025-09-12",215.000,"CALL"
"AAPL","2025-09-26",215.000,"PUT"
"AAPL","2025-09-26",215.000,"CALL"
"AAPL","2025-11-21",215.000,"CALL"
"AAPL","2025-11-21",215.000,"PUT"
"AAPL","2026-12-18",340.000,"CALL"
"AAPL","2025-12-19",215.000,"CALL"
"AAPL","2025-12-19",215.000,"PUT"
"AAPL","2027-01-15",340.000,"CALL"
"AAPL","2026-01-16",215.000,"CALL"
"AAPL","2026-01-16",215.000,"PUT"
"AAPL","2025-08-29",217.500,"PUT"
"AAPL","2025-08-29",217.500,"CALL"
"AAPL","2026-12-18",210.000,"PUT"
"AAPL","2026-12-18",210.000,"CALL"
"AAPL","2027-01-15",210.000,"CALL"
"AAPL","2027-01-15",210.000,"PUT"
"AAPL","2026-01-16",85.000,"PUT"
"AAPL","2027-12-17",205.000,"CALL"
"AAPL","2027-12-17",205.000,"PUT"
"AAPL","2026-12-18",80.000,"CALL"
"AAPL","2027-01-15",80.000,"PUT"
"AAPL","2025-08-29",220.000,"CALL"
"AAPL","2025-08-29",220.000,"PUT"
"AAPL",

# Function Defintions

In [5]:
# Extract the date from a filename where the date is in the format YYYYMMDD convert to datetime
def extract_date_from_filename_datetime(filename):
    date = re.search(r'\d{8}', filename)
    if date:
        return datetime.strptime(date.group(0), '%Y%m%d')
    return None

# Convert an integer representing a time in the format HHMMSS to a time object
def int_to_time(time_int):
    time_str = str(time_int).zfill(6)
    hour = int(time_str[:2])
    minute = int(time_str[2:4])
    second = int(time_str[4:])
    return time(hour=hour, minute=minute, second=second)

def event_time_to_burst_filename(event_dt):
    return f"Burst_{event_dt.strftime('%Y%m%d_%H%M%S')}.csv"

def get_symbol_burst_data(event_dt):
    filename = event_time_to_burst_filename(event_dt)
    filepath = os.path.join("./Data_Burst_Symbols", filename)
    # print(f"Looking for file: {filepath}")
    if os.path.exists(filepath):
        df = pl.read_csv(filepath, has_header=True)
        df = df.rename({
            df.columns[0]: "Symbol",
            df.columns[1]: "NumTrades",
            df.columns[2]: "NumQuotes",
            df.columns[3]: "BBO"
        })
        # Validate and clean the Symbol column
        df = df.with_columns([
            pl.col("Symbol").fill_null("").cast(pl.String)  # Replace nulls and cast to string
        ])
        # Convert rows to Polars-compatible List(Struct)
        return [
            {"Symbol": row["Symbol"], "NumTrades": row["NumTrades"], "NumQuotes": row["NumQuotes"], "BBO": row["BBO"]}
            for row in df.to_dicts()
        ]
    else:
        # Return a Polars-compatible List(Struct) with a single default value
        return [
            {"Symbol": "", "NumTrades": 0, "NumQuotes": 0, "BBO": 0}
        ]

# Initialize Parameters

In [12]:

print(f"Current working directory: {os.getcwd()}")

today_str = datetime.datetime.today().strftime('%Y%m%d')
print(f"Today's date string: {today_str}")

# make today-str year/month/day format
today_str1 = datetime.datetime.today().strftime('%Y/%m/')
print(f"Today's date string in year/month/day format: {today_str1}{today_str}")

csv_path = f"./Code/Data/{today_str}_hft1.csv"
csv_path_2 = f"./Data/{today_str}_hft2.csv"
csv_path_3 = f"./Data/{today_str}_hft3.csv"
burst_path = f"./Data_Burst/{today_str}.csv"
symbol_path = glob.glob(f"./Data_Burst_Symbols/Burst_{today_str}_??????.csv")
print (f"Symbol files: {symbol_path}")
print(f"Today's csv_path: {csv_path}")
parquet_path = f"./Processed_Data/{today_str1}{today_str}_hft1.parquet"
print(f"Today's parquet_path: {parquet_path}")
last_size_path = f"./Code/Data/{today_str}_hft1_lastsize.pkl"

Current working directory: c:\Users\vande\Desktop\Projects\Ki2_Alerts\Code
Today's date string: 20250827
Today's date string in year/month/day format: 2025/08/20250827
Symbol files: []
Today's csv_path: ./Code/Data/20250827_hft1.csv
Today's parquet_path: ./Processed_Data/2025/08/20250827_hft1.parquet


In [13]:
burst_symbol_files = glob.glob(f"./Data_Burst_Symbols/Burst_????????_??????.csv")
data_burst_files = glob.glob(f"./Data_Burst/????????.csv")
print(f'Found {len(burst_symbol_files)} burst symbol files')
print(f'Found {len(data_burst_files)} data burst files')
data_burst_files


Found 4805 burst symbol files
Found 21 data burst files


['./Data_Burst\\20250722.csv',
 './Data_Burst\\20250723.csv',
 './Data_Burst\\20250724.csv',
 './Data_Burst\\20250725.csv',
 './Data_Burst\\20250728.csv',
 './Data_Burst\\20250729.csv',
 './Data_Burst\\20250730.csv',
 './Data_Burst\\20250731.csv',
 './Data_Burst\\20250801.csv',
 './Data_Burst\\20250804.csv',
 './Data_Burst\\20250805.csv',
 './Data_Burst\\20250806.csv',
 './Data_Burst\\20250807.csv',
 './Data_Burst\\20250808.csv',
 './Data_Burst\\20250811.csv',
 './Data_Burst\\20250812.csv',
 './Data_Burst\\20250813.csv',
 './Data_Burst\\20250815.csv',
 './Data_Burst\\20250819.csv',
 './Data_Burst\\20250820.csv',
 './Data_Burst\\20250821.csv']

In [ ]:
import re
from datetime import datetime

def event_time_to_filename(event_dt):
    """
    Convert an event datetime to a filename format.
    Assumes event_dt is a datetime object.
    """
    return f"Burst_{event_dt.strftime('%Y%m%d_%H%M%S')}.csv"

def get_symbol_burst_data(event_dt):
    filename = event_time_to_filename(event_dt)
    filepath = os.path.join("./Data_Burst_Symbols", filename)
    # print(f"Looking for file: {filepath}")
    if os.path.exists(filepath):
        df = pl.read_csv(filepath, has_header=True)
        df = df.rename({
            df.columns[0]: "Symbol",
            df.columns[1]: "NumTrades",
            df.columns[2]: "NumQuotes",
            df.columns[3]: "BBO"
        })
        # Validate and clean the Symbol column
        df = df.with_columns([
            pl.col("Symbol").fill_null("").cast(pl.String)  # Replace nulls and cast to string
        ])
        # Convert rows to Polars-compatible List(Struct)
        return [
            {"Symbol": row["Symbol"], "NumTrades": row["NumTrades"], "NumQuotes": row["NumQuotes"], "BBO": row["BBO"]}
            for row in df.to_dicts()
        ]
    else:
        # Return a Polars-compatible List(Struct) with a single default value
        return [{"Symbol": "", "NumTrades": 0, "NumQuotes": 0, "BBO": 0}]
    
def add_external_data_for_symbols(df):
    #extract
    # This function will retrieve additional data for the specified symbols from thetadata using

    # http://127.0.0.1:25510/v2/hist/stock/ohlc?root=AAPL&start_date=20240102&end_date=20240102&ivl=1000&start_time=ms_of_day&end_time=ms_of_day+1000
    # http://127.0.0.1:25510/v2/hist/stock/trades?root=AAPL&start_date=20240102&end_date=20240102&ivl=1000&start_time=ms_of_day&end_time=ms_of_day+1000
    # http://127.0.0.1:25510/v2/hist/stock/quotes?root=AAPL&start_date=20240102&end_date=20240102&ivl=1000&start_time=ms_of_day&end_time=ms_of_day+1000

    #https://http-docs.thetadata.us/operations/get-v2-hist-stock-ohlc.html
    #https://http-docs.thetadata.us/operations/get-v2-hist-stock-trade.html
    #https://http-docs.thetadata.us/operations/get-v2-hist-stock-quote.html

    #integrate new data in struct
    # for symbol in burst_data:
    #     symbol_struct = pl.Struct([
    #         pl.Field("Symbol", pl.String),
    #         pl.Field("NumTrades", pl.Int64),
    #         pl.Field("NumQuotes", pl.Int64),
    #         pl.Field("BBO", pl.Int64),
    #         pl.Field("OHLC", pl.List(pl.Float64)),
    #         pl.Field("Trades", pl.List(pl.Float64)),
    #         pl.Field("Quotes", pl.List(pl.Float64))
    #     ])

    pass

# Define the struct for the symbol burst data
symbol_struct = pl.Struct([
    pl.Field("Symbol", pl.String),
    pl.Field("NumTrades", pl.Int64),
    pl.Field("NumQuotes", pl.Int64),
    pl.Field("BBO", pl.Int64)
])
    
#Read a Burst_Data file and convert to Polars Dataframe
test_df = pl.read_csv(data_burst_files[0], has_header=False)
test_df = test_df.rename({
    'column_1': 'Event_Date_Time',
    'column_2': 'Grade',
    'column_3': 'Prog_Type',
    'column_4': 'Bckgrnd',
    'column_5': 'S1',
    'column_6': 'S2',
    'column_7': 'CT',
    'column_8': 'CS',
    'column_9': 'CB'
})
#Extract Date from file name
file_name = data_burst_files[0].split('/')[-1]
print(f'Processing File: {file_name}')

# Extract the date part (YYYYMMDD) from the filename
date_match = re.search(r'(\d{8})', file_name)
if date_match:
    date_str = date_match.group(1)
    date_obj = datetime.strptime(date_str, "%Y%m%d")
    print(f'Extracted Date: {date_str}')
    print(f'Date as datetime: {date_obj}')
else:
    print("No date found in filename.")
print(f'Extracted Date: {date_str}')

test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(int_to_time, return_dtype=pl.Time).alias('Event_Date_Time'))
test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(lambda x: datetime.combine(date_obj, x), return_dtype=pl.Datetime).alias('Event_Date_Time'))

# Apply the function to the DataFrame
test_df = test_df.with_columns([
    pl.col("Event_Date_Time").map_elements(
        get_symbol_burst_data,
        return_dtype=pl.List(symbol_struct)
    ).alias("Burst_Symbols")
])

# Add a new column with ms_of_day based on Event_Date_Time
def get_ms_of_day(dt):
    return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + int(dt.microsecond / 1000)

test_df = test_df.with_columns(
    pl.col("Event_Date_Time").map_elements(get_ms_of_day, return_dtype=pl.Int64).alias("ms_of_day")
)

# http://127.0.0.1:25510/v2/hist/stock/trade?root=AAPL&start_date=20240102&end_date=20240102

# http://127.0.0.1:25510/v2/hist/stock/ohlc?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000

# http://127.0.0.1:25510/v2/hist/stock/quote?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000


# save parquet file to processed data directory
# year = date_obj.strftime("%Y")
# month = date_obj.strftime("%m")
# output_dir = f"./Processed_Data/{year}/{month}"
# os.makedirs(output_dir, exist_ok=True)
# output_path = f"{output_dir}/{date_str}_burst_symbols.parquet"
# test_df.write_parquet(output_path)
test_df

Processing File: Data_Burst\20250722.csv
Extracted Date: 20250722
Date as datetime: 2025-07-22 00:00:00
Extracted Date: 20250722


Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64
2025-07-22 09:30:01,2,0,0,149,296,1,0,0,"[{""AAPD"",0,188,49}, {""AAPL"",559,881,221}, … {""URTY"",6,865,143}]",34201000
2025-07-22 09:30:11,3,0,0,105,172,5,1,1,"[{""AAPB"",1,457,105}, {""AAPD"",0,454,97}, … {""YINN"",120,524,196}]",34211000
2025-07-22 09:30:14,3,0,0,79,107,6,2,1,"[{""AMD"",37,181,75}, {""AMDD"",0,998,181}, … {""ZETA"",53,302,73}]",34214000
2025-07-22 09:30:31,3,1,0,89,222,7,2,2,"[{""AAPB"",2,263,47}, {""AAPD"",0,375,127}, … {""XRT"",88,480,133}]",34231000
2025-07-22 09:31:05,3,1,0,84,216,8,2,3,"[{""AAPB"",6,214,36}, {""AAPD"",0,258,78}, … {""XRT"",20,649,181}]",34265000
…,…,…,…,…,…,…,…,…,…,…
2025-07-22 15:59:20,3,0,0,79,273,366,213,150,"[{""AAP"",108,294,87}, {""AAPL"",113,498,154}, … {""ZTO"",90,346,81}]",57560000
2025-07-22 15:59:56,3,0,0,138,292,367,214,150,"[{""AAL"",33,83,12}, {""AAPB"",2,303,87}, … {""YINN"",15,266,70}]",57596000
2025-07-22 15:59:58,3,1,0,76,247,368,214,151,"[{""AAPL"",108,502,174}, {""AAPU"",0,232,49}, … {""ZGN"",113,326,65}]",57598000


In [ ]:
def process_burst_file(file_path):
    #Read a Burst_Data file and convert to Polars Dataframe
    df = pl.read_csv(file_path, has_header=False)
    df = df.rename({
        'column_1': 'Event_Date_Time',
        'column_2': 'Grade',
        'column_3': 'Prog_Type',
        'column_4': 'Bckgrnd',
        'column_5': 'S1',
        'column_6': 'S2',
        'column_7': 'CT',
        'column_8': 'CS',
        'column_9': 'CB'
    })
    #Extract Date from file name
    file_name = file_path.split('/')[-1]
    print(f'Processing File: {file_name}')

    # Extract the date part (YYYYMMDD) from the filename
    date_match = re.search(r'(\d{8})', file_name)
    if date_match:
        date_str = date_match.group(1)
        date_obj = datetime.strptime(date_str, "%Y%m%d")
        print(f'Extracted Date: {date_str}')
        print(f'Date as datetime: {date_obj}')
    else:
        print("No date found in filename.")
        return None
    print(f'Extracted Date: {date_str}')

    df = df.with_columns(pl.col('Event_Date_Time').map_elements(int_to_time, return_dtype=pl.Time).alias('Event_Date_Time'))
    df = df.with_columns(pl.col('Event_Date_Time').map_elements(lambda x: datetime.combine(date_obj, x), return_dtype=pl.Datetime).alias('Event_Date_Time'))

    # Apply the function to the DataFrame
    df = df.with_columns([
        pl.col("Event_Date_Time").map_elements(
            get_symbol_burst_data,
            return_dtype=pl.List(symbol_struct)
        ).alias("Burst_Symbols")
    ])

    # Add a new column with ms_of_day based on Event_Date_Time
    def get_ms_of_day(dt):
        return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + int(dt.microsecond / 1000)

    df = df.with_columns(
        pl.col("Event_Date_Time").map_elements(get_ms_of_day, return_dtype=pl.Int64).alias("ms_of_day")
    )

    return df

In [22]:
def process_burst_file(file_path):

    # Define the struct for the symbol burst data
    symbol_struct = pl.Struct([
        pl.Field("Symbol", pl.String),
        pl.Field("NumTrades", pl.Int64),
        pl.Field("NumQuotes", pl.Int64),
        pl.Field("BBO", pl.Int64)
    ])
    
    #Read a Burst_Data file and convert to Polars Dataframe
    test_df = pl.read_csv(file_path, has_header=False)
    test_df = test_df.rename({
        'column_1': 'Event_Date_Time',
        'column_2': 'Grade',
        'column_3': 'Prog_Type',
        'column_4': 'Bckgrnd',
        'column_5': 'S1',
        'column_6': 'S2',
        'column_7': 'CT',
        'column_8': 'CS',
        'column_9': 'CB'
    })
    #Extract Date from file name
    file_name = file_path.split('/')[-1]
    print(f'Processing File: {file_name}')

    # Extract the date part (YYYYMMDD) from the filename
    date_match = re.search(r'(\d{8})', file_name)
    if date_match:
        date_str = date_match.group(1)
        date_obj = datetime.strptime(date_str, "%Y%m%d")
        print(f'Extracted Date: {date_str}')
        print(f'Date as datetime: {date_obj}')
    else:
        print("No date found in filename.")
    print(f'Extracted Date: {date_str}')

    test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(int_to_time, return_dtype=pl.Time).alias('Event_Date_Time'))
    test_df = test_df.with_columns(pl.col('Event_Date_Time').map_elements(lambda x: datetime.combine(date_obj, x), return_dtype=pl.Datetime).alias('Event_Date_Time'))

    # Apply the function to the DataFrame
    test_df = test_df.with_columns([
        pl.col("Event_Date_Time").map_elements(
            get_symbol_burst_data,
            return_dtype=pl.List(symbol_struct)
        ).alias("Burst_Symbols")
    ])

    # Add a new column with ms_of_day based on Event_Date_Time
    def get_ms_of_day(dt):
        return (dt.hour * 3600 + dt.minute * 60 + dt.second) * 1000 + int(dt.microsecond / 1000)

    test_df = test_df.with_columns(
        pl.col("Event_Date_Time").map_elements(get_ms_of_day, return_dtype=pl.Int64).alias("ms_of_day")
    )

    # http://127.0.0.1:25510/v2/hist/stock/trade?root=AAPL&start_date=20240102&end_date=20240102

    # http://127.0.0.1:25510/v2/hist/stock/ohlc?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000

    # http://127.0.0.1:25510/v2/hist/stock/quote?root=AAPL&start_date=20240102&end_date=20240102&ivl=60000


    # save parquet file to processed data directory
    # year = date_obj.strftime("%Y")
    # month = date_obj.strftime("%m")
    # output_dir = f"./Processed_Data/{year}/{month}"
    # os.makedirs(output_dir, exist_ok=True)
    # output_path = f"{output_dir}/{date_str}_burst_symbols.parquet"
    # test_df.write_parquet(output_path)
    return test_df

file_path = data_burst_files[1]

process_burst_file(file_path)

Processing File: Data_Burst\20250723.csv
Extracted Date: 20250723
Date as datetime: 2025-07-23 00:00:00
Extracted Date: 20250723


Event_Date_Time,Grade,Prog_Type,Bckgrnd,S1,S2,CT,CS,CB,Burst_Symbols,ms_of_day
datetime[μs],i64,i64,i64,i64,i64,i64,i64,i64,list[struct[4]],i64
2025-07-23 09:30:01,3,0,0,117,300,1,0,0,"[{""AAL"",66,251,54}, {""AAPL"",227,253,45}, … {""ZSL"",13,286,49}]",34201000
2025-07-23 09:30:02,2,0,0,256,590,2,0,0,"[{""AAPB"",0,460,95}, {""AAPD"",4,406,86}, … {""ZIPP"",0,182,44}]",34202000
2025-07-23 09:30:03,3,0,0,129,394,3,0,0,"[{""AAPD"",2,350,78}, {""AAPL"",749,611,217}, … {""ZETA"",50,277,71}]",34203000
2025-07-23 09:30:11,3,0,0,126,261,4,1,0,"[{""AAPB"",0,171,32}, {""AAPD"",0,985,274}, … {""ZETA"",57,175,43}]",34211000
2025-07-23 09:30:14,3,0,0,112,141,5,2,0,"[{""AAPD"",1,976,244}, {""AAPL"",226,309,110}, … {""XRX"",263,692,221}]",34214000
…,…,…,…,…,…,…,…,…,…,…
2025-07-23 15:59:57,3,1,0,90,197,174,100,71,"[{""ABR"",30,170,24}, {""ACWI"",7,239,96}, … {""XOP"",395,870,207}]",57597000
2025-07-23 15:59:58,3,1,0,93,257,175,100,72,"[{""AAPU"",1,235,55}, {""ACWI"",13,274,89}, … {""XYZ"",41,159,39}]",57598000
2025-07-23 15:59:59,3,1,0,125,292,176,100,73,"[{""AAPU"",6,316,101}, {""ACWI"",6,416,107}, … {""YINN"",15,181,27}]",57599000


In [19]:
def add_external_data_for_symbols(df):
    # Extract Burst_Symbol SStructure from df
    burst_data = df["Burst_Symbols"].explode().to_list()

    # Convert burst_data to polars dataframe
    burst_df = pl.DataFrame(burst_data)
    # This function will retrieve additional data for the specified symbols from thetadata using

    # http://127.0.0.1:25510/v2/hist/stock/ohlc?root=AAPL&start_date=20240102&end_date=20240102&ivl=1000&start_time=ms_of_day&end_time=ms_of_day+1000
    # http://127.0.0.1:25510/v2/hist/stock/trades?root=AAPL&start_date=20240102&end_date=20240102&ivl=1000&start_time=ms_of_day&end_time=ms_of_day+1000
    # http://127.0.0.1:25510/v2/hist/stock/quotes?root=AAPL&start_date=20240102&end_date=20240102&ivl=1000&start_time=ms_of_day&end_time=ms_of_day+1000

    #https://http-docs.thetadata.us/operations/get-v2-hist-stock-ohlc.html
    #https://http-docs.thetadata.us/operations/get-v2-hist-stock-trade.html
    #https://http-docs.thetadata.us/operations/get-v2-hist-stock-quote.html

    #integrate new data in struct
    # for symbol in burst_data:
    #     symbol_struct = pl.Struct([
    #         pl.Field("Symbol", pl.String),
    #         pl.Field("NumTrades", pl.Int64),
    #         pl.Field("NumQuotes", pl.Int64),
    #         pl.Field("BBO", pl.Int64),
    #         pl.Field("OHLC", pl.List(pl.Float64)),
    #         pl.Field("Trades", pl.List(pl.Float64)),
    #         pl.Field("Quotes", pl.List(pl.Float64))
    #     ])

    return burst_df

add_external_data_for_symbols(test_df)

Symbol,NumTrades,NumQuotes,BBO
str,i64,i64,i64
"""AAPD""",0,188,49
"""AAPL""",559,881,221
"""AAPU""",18,296,35
"""ABAT""",368,262,77
"""ACHR""",99,171,52
…,…,…,…
"""XYLD""",0,185,24
"""XYZ""",118,226,81
"""XYZG""",6,217,68
